In [1]:
import cv2


In [2]:
import mediapipe as mp
import numpy as np
from scipy.signal import lfilter
import matplotlib.pyplot as plt

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [4]:
hands = mp_hands.Hands(max_num_hands = 2, static_image_mode = False,min_detection_confidence = 0.5)

In [5]:
# storing all the coordinates at different timesteps in a list
list_x = []
list_y = []
list_z = []
list_avg_x = []
list_avg_y = []
list_avg_z = []


In [6]:
# color and thickness of the lines
color = (255,0,0)
thickness = 2
# threshold after which we should start drawing
threshold = 4

# moving average buffer
k = 5


In [7]:
# canvas on which we are drawing
canvas = np.zeros([512,512,1],dtype=np.uint8)


In [8]:
toggle = 1

     
    

In [9]:
cap = cv2.VideoCapture(0)
count = 0
while cap.isOpened():
    success, image = cap.read()
    
        
    if not success:
        print("video capture unsuccessful")
        break
    
    image = cv2.cvtColor(cv2.flip(image,1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    
    
    
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    key = cv2.waitKey(5)
    
    if key == 225:
        toggle = toggle^1
    elif key & 0xFF == ord('s'):
        break
    
    
        
    
    if results.multi_hand_landmarks:
        
        
        
            count = count+1
            hand_landmarks = results.multi_hand_landmarks[0]
        
            # add the coordinates to the list
            list_x.append(int(hand_landmarks.landmark[8].x*512))
            list_y.append(int(hand_landmarks.landmark[8].y*512))
            list_z.append(int(hand_landmarks.landmark[8].z*512))
            
            if(count == k):
                avg_x = sum(list_x[-k:])/k
                avg_y = sum(list_y[-k:])/k
                avg_z = sum(list_z[-k:])/k
            
                list_avg_x.append(int(avg_x))
                list_avg_y.append(int(avg_y))
                list_avg_z.append(int(avg_z))
                count = 0
            
        
            if(len(list_avg_x) > threshold and toggle == 1):
                # get the current coordinates as endpoint
                end_point = (list_avg_x[-1], list_avg_y[-1])
                # get the previous coordinates as the start point
                start_point = (list_avg_x[-2], list_avg_y[-2])
                # draw a line joining these points
                cv2.line(canvas,start_point, end_point,color, thickness)
                
                
                
       
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
            
    # show the canvas and the webcam image   
    cv2.imshow("Detected Hands", image)
    cv2.imshow("Result", canvas)
    
    
    
  
hands.close()
cv2.destroyAllWindows()
cap.release()






In [10]:
print(list_avg_x)

[299, 274, 272, 277, 283, 285, 284, 284, 284, 286, 285, 285, 285, 285, 284, 284, 285, 291, 339, 298, 303, 306, 315, 329, 339, 316, 233, 147, 131, 143, 156, 160, 147, 145, 148, 150, 148, 153, 150, 143, 153, 149, 164, 167, 315, 301, 281, 220, 213, 133, 101, 117, 139, 154, 178, 181, 182, 183, 193, 198, 213, 227, 246, 273, 285, 280, 215, 183, 178, 162, 153, 153, 155, 154, 152, 152, 171, 231, 290, 340, 377, 331, 301, 310, 312, 308]


In [11]:
print(list_avg_y)

[308, 182, 157, 150, 149, 147, 149, 145, 148, 145, 145, 149, 145, 146, 147, 151, 171, 220, 298, 262, 213, 197, 191, 190, 204, 226, 299, 197, 173, 168, 160, 157, 153, 153, 158, 160, 165, 170, 164, 202, 163, 185, 258, 246, 277, 297, 170, 138, 210, 297, 373, 422, 423, 424, 422, 411, 411, 415, 404, 375, 345, 347, 352, 369, 390, 364, 312, 298, 298, 268, 262, 261, 261, 263, 264, 269, 270, 285, 303, 273, 279, 231, 165, 141, 137, 144]


In [12]:
print(count)

1


In [13]:
print(len(list_avg_x))

86
